In [3]:
#import libraries into python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

In [4]:
#load data set into python
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc','id','member_id'], 1, inplace=True)
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [6]:
#set the target variable and features
x = y2015.drop('loan_status', 1)
y = y2015['loan_status']
x = pd.get_dummies(x, drop_first=True)
x = x.dropna(axis=1)


In [83]:
#create a train and test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [84]:
#initialize the random forest module and get the score
rfc = ensemble.RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_test, y_test)

0.9813937472541826

In [85]:
#get the cross value score to check for overfitting
cross_val_score(rfc, x, y, cv=5)

array([0.98081193, 0.98166684, 0.83730512, 0.76814021, 0.9808713 ])

In [7]:
#create copy of the dataset
df = y2015.dropna(axis=1)

In [8]:
#create a copy of the target variable and features
x2 = df.drop('loan_status', 1)
y2 = df['loan_status']
x2 = pd.get_dummies(x2, drop_first=True)
x2 = x2.dropna(axis=1)

In [9]:
#create a copy of the train/test splits
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size = 0.2)

In [10]:
#perform standard scaler on the numerical feature columns
scaler = StandardScaler()
categorical = df.select_dtypes(include=['object'])
numerical = df.select_dtypes(include=['float'])
numeric_col = numerical.columns
x_train2[numeric_col] = scaler.fit_transform(x_train2[numeric_col])
x_test2[numeric_col] = scaler.transform(x_test2[numeric_col])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [90]:
#initialize the random forest model and get the cross value score for the scaled features
rfc2 = ensemble.RandomForestClassifier()
rfc2.fit(x_train2, y_train2)
cross_val_score(rfc2, x2, y2, cv=5)

array([0.93752004, 0.95986654, 0.69296715, 0.74438072, 0.9605196 ])

In [91]:
#get the accuracy of the model based on the test split
rfc2.score(x_test2, y_test2)

0.9625025231836046

In [11]:
#perform dimension reduction on the features via PCA
pca = PCA(n_components = 10)
pca.fit(x_train2)
x_train_pca = pca.transform(x_train2)
x_test_pca = pca.transform(x_test2)

In [13]:
#initialize the random forest model and scores for the PCA-scaled features
rfc_pca = ensemble.RandomForestClassifier()
rfc_pca.fit(x_train_pca, y_train2)
rfc_pca.score(x_test_pca, y_test2)

0.9259549507830774

In [14]:
#get the cross value score for PCA-scaled data
x_pca = np.concatenate([x_train_pca, x_test_pca], axis=0)
y_pca = np.concatenate([y_train2, y_test2], axis=0)
cross_val_score(rfc_pca, x_pca, y_pca, cv=3)

array([0.92446123, 0.9234852 , 0.92382716])

In [95]:
#initialize the neural network and get the accuracy
mlp = MLPClassifier(hidden_layer_sizes=(100,20))
mlp.fit(x_train_pca, y_train2)
mlp.score(x_test_pca, y_test2)

0.9490613756990702

In [96]:
#get the cross value score to check for overfitting
cross_val_score(mlp, x_pca, y_pca, cv=3)

array([0.94957432, 0.94956008, 0.94894739])

In [97]:
#intialize the neural network and get the accuracy
mlp = MLPClassifier(hidden_layer_sizes=(50,50))
mlp.fit(x_train_pca, y_train2)
mlp.score(x_test_pca, y_test2)

0.9491326185302604

In [98]:
#get the cross value score and check for overfitting
cross_val_score(mlp, x_pca, y_pca, cv=3)

array([0.94926086, 0.94978805, 0.94964557])

In [99]:
#initialize the neural network and get the accuracy
mlp = MLPClassifier(hidden_layer_sizes=(100,50))
mlp.fit(x_train_pca, y_train2)
mlp.score(x_test_pca, y_test2)

0.9489307638418885

In [100]:
#get the cross value score and check for overfitting
cross_val_score(mlp, x_pca, y_pca, cv=3)

array([0.94931073, 0.94879778, 0.94904713])

In [16]:
#initialize neural network and get accuracy
mlp = MLPClassifier(hidden_layer_sizes=(50,50), activation ='logistic')
mlp.fit(x_train_pca, y_train2)
mlp.score(x_test_pca, y_test2)

0.9497975516213681

In [17]:
#get cross value score
cross_val_score(mlp, x_pca, y_pca, cv=3)

array([0.94919674, 0.94962419, 0.94914687])

In [103]:
#initialize neural network and get accuracy
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation ='logistic')
mlp.fit(x_train_pca, y_train2)
mlp.score(x_test_pca, y_test2)

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.9496075707381945

In [104]:
#get cross value score
cross_val_score(mlp, x_pca, y_pca, cv=3)

array([0.94968831, 0.94955295, 0.95001603])

After using this data set with a random forest model and a 2 layer neural network, the neural network gave better results. After varying the hyperparameter of the neural network, the averages were around 94.8 - 95% compared to the 92.3% of the random forest. While the accuracy of the neural network was better, the tradeoff was the complexity and time-consumption. The random forest was considerably faster to train and fit while also would have some explainability in the feature importance. This would at least tell us what features are influencing the model the most.